## Loading Packages

In [2]:
import os
import sys
import math

import numpy as np
import pandas as pd

from datetime import timedelta

# for plotting
import matplotlib.pyplot as plt

import shap

# progress bar
from tqdm import tqdm

# Download from
# https://github.com/dynalogic-agtech/dynalogic-agro-met-equations-pkg/tree/14c5f973676152f95005c1ecbcff423e3ea7ce28
import fao

import sklearn
from sklearn.model_selection import GridSearchCV, RepeatedKFold # train_test_split, RepeatedKFold, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# XGBoost
import xgboost
from xgboost import XGBRegressor
import seaborn as sns

import pickle

import matplotlib
# # updating the matplotlib configuration parameters
# mpl.rcParams.update({'font.size': 20, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Setting Directory

In [ ]:
# current working directory
path = os.getcwd()
print('Default path: {}'.format(path))

## Importing Data

In [ ]:
## Importing the csv from folder
# Columns: date	tmax	Smx	tmin	Smn	rain	Srn	Evap	Sev	radn	...	area_soybeans	area_canola	area_corn	area_almond	area_barley	area_oats	area_sorghum	area_oilseeds	area_pasture	net_diversion
# Data not provided here
df_actual_weather = pd.read_csv('daily_weather.csv')

df_actual_weather.head(-10)

## Preparing Dataset

In [ ]:
## Converting date format
df_actual_weather['date'] = pd.to_datetime(df_actual_weather['date'], format = '%d/%m/%Y')  ## %d/%m/%Y, %Y%m%d

# setting date as index
df_actual_weather.set_index('date', inplace = True)

df_actual_weather.head()

## Computing ETo

In [19]:
## Assign altitude, latitude, longitude, soil heat flux, and time period for ET calculation
altitude = 121
latitude_degree = -34.8016
soil_heat_flux = 0.0  # soil heat flux MJ/m^2/day default= 0.0
time_period = "daily"
wind_speed = 2  # wind speed m/s

In [ ]:
# converting from decimal degrees to radian
latitude = math.radians(latitude_degree)

for index, row in tqdm(df_actual_weather.iterrows(), total = df_actual_weather.shape[0]):
    # print(index, row)
    date = index.date()
    day_of_year = pd.to_datetime(date).dayofyear
    avp = df_actual_weather.loc[index, 'vp']/10 # kPa
    temperature_min = df_actual_weather.loc[index, 'tmin']  # oC
    temperature_max = df_actual_weather.loc[index, 'tmax'] # oC
    solar_radiation = df_actual_weather.loc[index, 'radn']  # MJ/m^2/day
    # wind_speed_3m =df_actual_weather.loc[index, 'wind_2m'] # km/h
    

    # Clear sky radiation
    solar_declination = fao.sol_dec(day_of_year)
    sunset_hour_angle = fao.sunset_hour_angle(latitude, solar_declination)
    inverse_relative_distance = fao.inv_rel_dist_earth_sun(day_of_year)
    extraterrestrial_radiation = fao.et_rad(latitude, solar_declination, sunset_hour_angle, inverse_relative_distance)
    clear_sky_radiation = fao.cs_rad(altitude, extraterrestrial_radiation)
    
    # Degree to Kelvin
    temperature_min_kelvin = temperature_min + 273
    temperature_max_kelvin = temperature_max + 273
    
    # Net outgoing long-wave, net incoming short-wave, net total radiations
    net_outgoing_longwave_radiation = fao.net_out_lw_rad(temperature_min_kelvin, temperature_max_kelvin, solar_radiation,
                                                         clear_sky_radiation, avp)
    net_incoming_shortwave_radiation = fao.net_in_sol_rad(solar_radiation, albedo=0.23)
    net_radiation = fao.net_rad(net_incoming_shortwave_radiation, net_outgoing_longwave_radiation)
    
    # Saturation vapour pressure
    saturation_vp_max = fao.svp_from_t(temperature_max)
    saturation_vp_min = fao.svp_from_t(temperature_min)
    sat_vp = fao.svp(saturation_vp_min, saturation_vp_max)
    
    temperature_mean = (temperature_min + temperature_max) / 2
    
    # Parameters
    latent_heat = fao.latent_heat(temperature_mean)
    delta_saturation_vp = fao.delta_svp(temperature_mean)
    # wind_speed = fao.wind_speed_2m(wind_speed_3m, 3)
    
    atmosphere_pressure = fao.atm_pressure(altitude)
    psy = fao.psy_const(atmosphere_pressure, latent_heat)
    
    eto = fao.fao56_penman_monteith(net_radiation, temperature_mean, wind_speed, latent_heat, sat_vp, avp,
                                    delta_saturation_vp, psy, soil_heat_flux, time_period)  # -> mm/day
    # print(date, eto)

    df_actual_weather.loc[index, 'eto'] = round(eto, 1)

## Crop Irrigation Requirement

    crop_evapotranspiration(etc) = crop_coefficients (kc)* reference evapotranspiration
    cwr = [{(etc (mm) - daily_eff_rain (mm))/1000000 (converting to km)] * Area (km2)}] * 1000000

### Computing crop evapotranspiration (ETc) for each crop

In [21]:
df_actual_weather['etc_rice'] = df_actual_weather['kc_rice'] * df_actual_weather['eto']
df_actual_weather['etc_cotton'] = df_actual_weather['kc_cotton'] * df_actual_weather['eto']
df_actual_weather['etc_winter_wheat'] = df_actual_weather['kc_winter_wheat'] * df_actual_weather['eto']
df_actual_weather['etc_soybeans'] = df_actual_weather['kc_soybeans'] * df_actual_weather['eto']
df_actual_weather['etc_canola'] = df_actual_weather['kc_canola'] * df_actual_weather['eto']
df_actual_weather['etc_corn'] = df_actual_weather['kc_corn'] * df_actual_weather['eto']
df_actual_weather['etc_almond'] = df_actual_weather['kc_almond'] * df_actual_weather['eto']
df_actual_weather['etc_barley'] = df_actual_weather['kc_barley'] * df_actual_weather['eto']
df_actual_weather['etc_oats'] = df_actual_weather['kc_oats'] * df_actual_weather['eto']
df_actual_weather['etc_sorghum'] = df_actual_weather['kc_sorghum'] * df_actual_weather['eto']
df_actual_weather['etc_oilseeds'] = df_actual_weather['kc_oilseeds'] * df_actual_weather['eto']
df_actual_weather['etc_pasture'] = df_actual_weather['kc_pasture'] * df_actual_weather['eto']

### Soil water balance

## Case 1
* Other assumptions
  * max soil moisture (s_max) = 100 mm
  * soil moisture threshold for irrigation (s_o) = 50 mm

In [22]:
# runoff
q = 0
# maximum soil moisture (100)
s_max = 100
# soil moisture threshold for irrigation (s_o < 50 implies irrigation is required) (s_o = s_max/2)
s_o = s_max/2

In [23]:
crops_list = ['rice', 'cotton', 'winter_wheat', 'soybeans', 'canola', 'corn', 'almond', 'barley', 'oats', 'sorghum', 'oilseeds', 'pasture']

# # total irrigation [case 1]
# df_cwr_rainfall['c1_total_irr'] = 0

# assuming the soil moisture equivalent to s_o 
s_initial_dict = dict()
for crop in crops_list:
    s_initial_dict[crop] = s_o

for index, row in df_actual_weather.iterrows(): # df_cwr_rainfall.iloc[595:].head(10)
    # print(index, row['rain'])

    total_irrigation = 0

    for crop in crops_list:
        area_column = 'area_' + crop
        etc_column = 'etc_' + crop
        s_column = 'c1_s_' + crop
        # irr_column = 'c1_irr_' + crop
        irr_column = 'CIR_' + crop
        runoff_column = 'c1_ro_' + crop
        # print('\t', crop, etc_column, s_column, irr_column)

        # computing crop specific soil moisture storage
        s_crop = (s_initial_dict[crop] + row['rain'] - row[etc_column]).round(3)
        
        # crop specific soil moisture greater than maximum soil moisture capacity 
        if s_crop > s_max:
            # computing runoff (assumption: no infiltration)
            runoff_crop = s_crop - s_max
            # irrigation
            irr_crop = 0
            # setting crop specific soil moisture equivalent to the maximum soil moisture capacity
            s_crop = s_max

            # updating dataframe
            df_actual_weather.loc[index, s_column] = s_crop
            df_actual_weather.loc[index, irr_column] = irr_crop
            df_actual_weather.loc[index, runoff_column] = runoff_crop

        # crop specific soil moisture between soil moisture threshold and maximum soil moisture capacity
        elif s_max/2 <= s_crop <= s_max:
            # runoff
            runoff_crop = 0
            # irrigation
            irr_crop = 0

            # updating dataframe
            df_actual_weather.loc[index, s_column] = s_crop
            df_actual_weather.loc[index, irr_column] = irr_crop
            df_actual_weather.loc[index, runoff_column] = runoff_crop
            
        # crop specific soil moisture less than soil moisture threshold
        else:
            # runoff
            runoff_crop = 0
            # irrigation
            irr_crop = s_max/2 - s_crop 
            # after irrigation s_crop will be equivalent to s_o (s_max/2)
            s_crop = s_max/2

            # updating dataframe
            df_actual_weather.loc[index, s_column] = s_crop
            df_actual_weather.loc[index, irr_column] = irr_crop
            df_actual_weather.loc[index, runoff_column] = runoff_crop
        
        # updating s_initial_dict
        s_initial_dict[crop] = s_crop

        # adding up irrigation to get total irrigation
        total_irrigation += (irr_crop/1000000 * row[area_column]*1000000).round(3)
    
    df_actual_weather.loc[index, 'CIR_total'] = total_irrigation

#### Visualization of CIR vs Actual Demand (ML)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))

# df_cwr_rainfall.plot(y = 'daily_cwr', ax = ax, legend = True, label = 'Daily CWR (ML)') # color = 'blue'
df_actual_weather.plot(y = 'CIR_total', ax = ax, legend = True, label = 'Daily CIR (ML)', color = 'green', fontsize= 10) # color = 'blue'
df_actual_weather.plot(y = 'net_diversion', ax = ax, legend = True, label = 'Actual Demand (ML)', color = 'orange', fontsize= 10)
# df_cwr_rainfall.plot(y = 'rain', ax = ax, legend = True, label = 'Rain (mm)', color = 'red')

# ax.set_ylim([0, 6000])

plt.title('Comparison Daily Actual Demand and CWN')
plt.xlabel('Date')
plt.ylabel('Water Demand (ML)')

plt.show()

In [ ]:
### Rain distribution throughout the year from 2018 to 2024
fig, ax = plt.subplots(figsize = (8, 5))
df_actual_weather.plot(y = 'rain', ax = ax, legend = True, label = 'Rain (mm)', color = 'red')

# ax.set_ylim([0, 6000])

plt.title('Rain')
plt.xlabel('Date')
plt.ylabel('Daily rain (mm)')

plt.show()

### Saving the data in csv

In [26]:
df_actual_weather.to_csv('./weather_and_CIR.csv')

## Data-driven Model

In [ ]:
### Reading csv file from the directory

df_diversion = pd.read_csv('weather_and_CIR.csv')


df_diversion['date'] = pd.to_datetime(df_diversion['date'], format = '%Y-%m-%d')
# df_diversion['date'] = pd.to_datetime(df_diversion['date'], format = '%d/%m/%Y')
# setting date as index
df_diversion.set_index('date', inplace = True)

df_diversion.head()

## Selecting the features

7 days lead and 7 days lag input features

In [ ]:
max_lag_days = 7

feature_list = ['tmin', 'tmax', 'rain', 'eto', 'radn', 'CIR_total', 'net_diversion']

df_features = pd.DataFrame(data = None)

for index, row in df_diversion[feature_list].iterrows():

    ######################### lag part (t-1, ..., t-7) #########################
    lag_start_date = index - timedelta(days = max_lag_days)
    lag_end_date = index - timedelta(days = 1)
    # print(index, lag_start_date, lag_end_date)

    temp_df = df_diversion.loc[lag_start_date:lag_end_date]

    if len(temp_df) == 0: # len(temp_df) != max_lag_days
        # print('Insufficient data')
        counter = max_lag_days
        for c in range(max_lag_days, 0, -1):
            for feature in feature_list:
                new_feat_name = feature + '_m{}'.format(c)
                df_features.loc[index, new_feat_name] = np.NaN

        # continue # for looping
    else:
        # print('Sufficient data')
        counter = len(temp_df)
        for index1, row1 in temp_df.iterrows():
            for feature in feature_list:
                new_feat_name = feature + '_m{}'.format(counter) 
                df_features.loc[index, new_feat_name] = row1[feature]
            
            counter = counter - 1


    ######################### t #########################
    for feature in feature_list:
        new_feat_name = feature + '_0'
        df_features.loc[index, new_feat_name] = row[feature]
    
    
    ######################### forecast part (t-1, ..., t-7) #########################
    forecast_start_date = index + timedelta(days = 1)
    forecast_end_date = index + timedelta(days = max_lag_days)
    # print(index, forecast_start_date, forecast_end_date)

    temp_df = df_diversion.loc[forecast_start_date:forecast_end_date]

    if len(temp_df) == 0: # len(temp_df) != max_lag_days
        # print('Insufficient data')
        counter = max_lag_days
        for c in range(1, counter + 1):
            for feature in feature_list:
                new_feat_name = feature + '_p{}'.format(c) 
                df_features.loc[index, new_feat_name] = np.NaN
        # continue # for looping
    else:
        # print('Sufficient data')
        counter = 1
        for index1, row1 in temp_df.iterrows():
            for feature in feature_list:
                new_feat_name = feature + '_p{}'.format(counter) 
                df_features.loc[index, new_feat_name] = row1[feature]
            
            counter = counter + 1

print(df_features.head(10))

In [29]:
### Saving the features as csv
df_features.to_csv('./feature_dataset.csv')

## All required dataset

In [30]:
# data with actual diversion
csv_diversion = 'weather_and_CIR.csv'

# feature data
csv_feature = 'feature_dataset.csv'

### Reading the feature data

In [ ]:
df_feature = pd.read_csv(csv_feature)
df_feature.head()

## Rename the first column as date

In [32]:
df_feature.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
df_feature.to_csv(csv_feature, header = True, index = False, sep = ',')

In [ ]:
# setting date as index
df_feature['date'] = pd.to_datetime(df_feature['date'], format = '%Y-%m-%d')

# setting date as index
df_feature.set_index('date', inplace = True)

df_feature.head()

## Adding actual diversion column from diversion dataframe to feature dataframe

In [ ]:
diversion_column = 'net_diversion'
if not diversion_column in df_feature.columns:
    df_feature = df_feature.merge(df_diversion['net_diversion'], how = 'inner', on = 'date')

df_feature.head()

## Filtering data for Training and Testing

In [ ]:
start_year = 2019
end_year = 2022

df_feature_subset = df_feature[(df_feature.index.year >= start_year) & (df_feature.index.year <= end_year)]
df_feature_subset.head()

# Leave one year out validation
* K-fold Cross-Validation

  ![k-fold cross-validation](https://miro.medium.com/v2/resize:fit:720/format:webp/1*RWjVdxk-pcdoGa02t8MLWg.png)

* Time Series Split Cross-Validation

  ![time series split cross-validation](https://miro.medium.com/v2/resize:fit:640/format:webp/1*XcqvKVTQ6U_zszSD52lSqA.png)

* Blocked Cross-Validation

  ![blocked cross-validation](https://miro.medium.com/v2/resize:fit:640/format:webp/1*QJaeOqGfe_vKbpmT882APA.png)

In [37]:
seed = 42
np.random.seed =seed

### Selecting feature based on lead and lag days based on the defined criterion

* m = 7 lag days inputs
* p = 7 lead days inputs
* mp = combining 7 days lead and lag inputs

In [38]:
def select_variables(col_list, filter_col_list, criterion = 'all'):
    '''
    criterion: 'all', 'm', 'p', 'mp'           
    '''
    var_list = list()
    for fc in filter_col_list:
        if criterion == 'all':
            l = [col for col in col_list if col.startswith(fc)]
            var_list += l

        elif criterion == 'm':
            fc = fc + '_m'
            l = [col for col in col_list if col.startswith(fc)]
            var_list += l

        elif criterion == 'p':
            fc = fc + '_p'
            l = [col for col in col_list if col.startswith(fc)]
            var_list += l

        elif criterion == 'mp':
            fc1 = fc + '_m'
            fc2 = fc + '_p'
            l = [col for col in col_list if (col.startswith(fc1) or col.startswith(fc2))]
            var_list += l

    return var_list

## XGBoost
The most commonly configured hyperparameters are the following:
* *n_estimators*: The number of trees in the ensemble, often increased until no further improvements are seen.
* *max_depth*: The maximum depth of each tree, often values are between 1 and 10.
* *learning_rate*: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.
* *subsample*: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.
* *colsample_bytree*: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.

https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor

In [ ]:
weather_vars = ['tmin', 'tmax', 'rain', 'radn', 'eto']
other_vars = ['CIR_total', 'net_diversion']

w_col = select_variables(list(df_feature_subset.columns), weather_vars, criterion = 'm') # criterion: 'all', 'm', 'p', 'mp'
o_col = select_variables(list(df_feature_subset.columns), other_vars, criterion = 'm') # criterion: 'all', 'm','p', 'mp'

year_list = list(range(start_year, end_year + 1))
# print(year_list)

rmse_list = list()
nse_list = list()
acc_list = list()
msss_list = list()
r2_list = list()

for year in year_list:
    print('Train: {} | Test: {}'.format('--', year))
    # training set
    x_train = df_feature_subset[df_feature_subset.index.year != year][w_col + o_col]
    y_train = df_feature_subset[df_feature_subset.index.year != year][diversion_column]

    # testing set
    x_test = df_feature_subset[df_feature_subset.index.year == year][w_col + o_col]
    y_test = df_feature_subset[df_feature_subset.index.year == year][diversion_column]

    # print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

    # creating an xgboost regression model
    # xgb_model = XGBRegressor(random_state = seed)
    xgb_model = XGBRegressor(n_estimators = 100, max_depth = 7, learning_rate = 0.1, subsample = 0.7, colsample_bytree = 0.8, random_state = seed)

    # fitting the model
    xgb_model.fit(x_train, y_train)

    # predicting on test set
    y_pred = xgb_model.predict(x_test)

    ## evaluating the model
    # rmse
    rmse = round(mean_squared_error(y_test, y_pred, squared = False), 3)

    # nash-sutcliffe efficienty (NSE) - equivalent to the coefficient of determination
    nse = round(1 - np.sum(np.power(y_pred - y_test, 2))/np.sum((np.power(y_test - np.mean(y_test), 2))), 4)

    # anomaly correlation coefficient (ACC)
    # sharpness of forecasting | ability to forecast extreme values
    mean_monthly_diversion = np.mean(df_feature_subset[diversion_column])
    acc = round(np.sum((y_pred - mean_monthly_diversion)*(y_test - mean_monthly_diversion))/np.sqrt(np.sum(np.power(y_pred - mean_monthly_diversion, 2))*np.sum(np.power(y_test - mean_monthly_diversion, 2))), 4)

    # mean square skill score (MSSS)
    # shows the forecaset skill by quantifying accuracy relative to the long-term monthly averged irrigation demand
    # value ranges from 0 to infinity
    # a good model will have an MSSS value closer to zero
    n = len(x_test)
    msss = round(1 - np.sum(np.power(y_pred - mean_monthly_diversion, 2))/((n/(n-1))*np.sum(np.power(y_test - mean_monthly_diversion, 2))), 4)

    # coefficient of determination
    r2 = round(r2_score(y_test, y_pred), 4)
    print('\tRMSE: {} | NSE: {} | ACC: {} | MSSS: {} | R-squared: {}'.format(rmse, nse, acc, msss, r2))

    rmse_list.append(rmse)
    nse_list.append(nse)
    acc_list.append(acc)
    msss_list.append(msss)
    r2_list.append(r2)

In [ ]:
print('''\nRMSE: {} \u00B1 {} | NSE: {} \u00B1 {} | ACC: {} \u00B1 {} | MSSS: {} \u00B1 {} | R-squared: {} \u00B1 {}'''.format(np.round(np.nanmean(rmse_list), 3), np.round(np.nanstd(rmse_list), 3),
                                                                                                                               np.round(np.nanmean(nse_list), 4), np.round(np.nanstd(nse_list), 4),
                                                                                                                               np.round(np.nanmean(acc_list), 4), np.round(np.nanstd(acc_list), 4),
                                                                                                                               np.round(np.nanmean(msss_list), 4), np.round(np.nanstd(msss_list), 4),
                                                                                                                               np.round(np.nanmean(r2_list), 4), np.round(np.nanstd(r2_list), 4),))

## Creating custom cross-validation
* https://nander.cc/writing-custom-cross-validation-methods-grid-search
* https://stackoverflow.com/questions/30040597/how-to-generate-a-custom-cross-validation-generator-in-scikit-learn
* https://domino.ai/blog/guide-to-building-models-with-cross-validation
* https://domino.ai/blog/guide-to-building-models-with-cross-validation

In [42]:
class CustomCrossValidation:
    '''
    Return the index of train and test set
    '''
    def __init__(self, start_year, end_year):
        self.start_year = start_year
        self.end_year = end_year
        self.n_splits = self.end_year - self.start_year + 1

    def split(self, X, y, groups = None):
        year_list = list(range(self.start_year, self.end_year + 1))
        for year in year_list:
            # training set index
            X_ = X.copy()
            X_['year'] = X_.index.year
            X_ = X_.reset_index(drop = True)
            train_index = X_[X_['year'] != year].index

            # testing set index
            test_index = X_[X_['year'] == year].index

            yield train_index, test_index

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

### XGBoost model
* Run only if the training model has to be updated

In [ ]:
weather_vars = ['tmin', 'tmax', 'rain', 'radn', 'eto']
other_vars = ['CIR_total', 'net_diversion']

w_col = select_variables(list(df_feature_subset.columns), weather_vars, criterion = 'm') # criterion: 'all', 'm', 'p','mp'
o_col = select_variables(list(df_feature_subset.columns), other_vars, criterion = 'm') # criterion: 'all', 'm', 'p', 'mp'

n_jobs = 8 # number of cpu
xgb_model = XGBRegressor(learning_rate = 0.1, subsample = 1, random_state = seed)

param_grid = {'n_estimators': [50, 100, 200], # [50, 100, 200]
              'max_depth': [2, 5, 10], # [2, 5, 10]
              'subsample': [0.7, 0.8, 0.9, 1.0], # the number of samples used in each tree
              'colsample_bytree': [0.7, 0.8, 0.9, 1.0], # number of feature used in each tree
              }

# splitter for cross-validation
# custom_splitter = CustomCrossValidation(start_year, end_year).split(X = df_feature_subset[w_col + o_col], y = df_feature_subset[diversion_column])

custom_splitter = CustomCrossValidation(start_year, end_year).split(X = df_feature_subset[w_col + o_col], 
                                                                    y = df_feature_subset['net_diversion_p7'])

# grid search
xgb_grid_model = GridSearchCV(xgb_model, param_grid = param_grid, cv = custom_splitter, n_jobs = n_jobs, verbose = 1) # cv = 5


xgb_grid_model.fit(df_feature_subset[w_col + o_col], df_feature_subset['net_diversion_p7'])

xgb_grid_model_best = xgb_grid_model.best_estimator_

print('n_estimators: {} | max_depth: {}'.format(xgb_grid_model_best.n_estimators, xgb_grid_model_best.max_depth))

### Saving the model
* If the model is retrained, it has to be saved in the folder accordingly.

In [ ]:
# saving the model
model_name = './model_lead_7.pkl'
print('Output model: {}'.format(model_name))
pickle.dump(xgb_grid_model_best, open(model_name, 'wb'))

# # # loading model
# xgb_grid_model_best = pickle.load(open(model_name, 'rb'))

## 7 days prediction at once

In [60]:
def evaluate_rmse(y_true, y_pred):
    return round(mean_squared_error(y_true, y_pred, squared = False), 3)

In [61]:
def evaluate_nse(y_true, y_pred):
    # nash-sutcliffe efficienty (NSE) - equivalent to the coefficient of determination
    return round(1 - np.sum(np.power(y_true - y_pred, 2))/np.sum((np.power(y_true - np.mean(y_true), 2))), 4)

In [62]:
def evaluate_acc(y_true, y_pred, mean_val):
    # anomaly correlation coefficient (ACC)
    # sharpness of forecasting | ability to forecast extreme values
    return round(np.sum((y_pred - mean_val)*(y_true - mean_val))/np.sqrt(np.sum(np.power(y_pred - mean_val, 2))*np.sum(np.power(y_true - mean_val, 2))), 4)

In [63]:
def evaluate_msss(y_true, y_pred, mean_val):
    # mean square skill score (MSSS)
    # shows the forecaset skill by quantifying accuracy relative to the long-term monthly averged irrigation demand
    # value ranges from 0 to infinity
    # a good model will have an MSSS value closer to zero
    n = len(y_true)
    return round(1 - np.sum(np.power(y_pred - mean_val, 2))/((n/(n-1))*np.sum(np.power(y_true - mean_val, 2))), 4)

In [64]:
def evaluate_r2(y_true, y_pred):
    # coefficient of determination
    return round(r2_score(y_true, y_pred), 4)

In [ ]:
weather_vars = ['tmin', 'tmax', 'rain', 'radn', 'eto']
other_vars = ['CIR_total', 'net_diversion']

w_col = select_variables(list(df_feature_subset.columns), weather_vars, criterion = 'm') # criterion: 'all', 'm', 'p', 'mp'
o_col = select_variables(list(df_feature_subset.columns), other_vars, criterion = 'm') # criterion: 'all', 'm', 'p', 'mp'

df_feature_train = df_feature[(df_feature.index.year >=2019) & (df_feature.index.year <= 2022)]
df_feature_test = df_feature[df_feature.index.year >= 2023]


min_lead_pred = 1
max_lead_pred = 7

for lead_day in range(min_lead_pred, max_lag_days + 1):
    print('Lead day: {}'.format(lead_day))

    # loading saved model
    model_name = './model_lead_{}.pkl'.format(lead_day)
    print('\tModel name: {}'.format(model_name))

    # loading model
    model = pickle.load(open(model_name, 'rb'))

    actual_diversion_colname = 'net_diversion_p{}'.format(lead_day)
    pred_diversion_colname = 'pred_diversion_p{}'.format(lead_day)
    print('\tActual diversion column name: {}'.format(actual_diversion_colname))
    print('\tPredicted diversion column name: {}'.format(pred_diversion_colname))

    ################## train set prediction ##################
    X_train = df_feature_train[w_col + o_col]
    y_train = df_feature_train[actual_diversion_colname]

    y_pred = model.predict(X_train)
    df_feature_train[pred_diversion_colname] = np.round(y_pred, 2)

    # Find indices of NaN values in y_test or y_pred
    nan_indices = np.isnan(y_train) | np.isnan(y_pred)

    # Filter out NaN values from both y_test and y_pred
    y_train = y_train[~nan_indices]
    y_pred = y_pred[~nan_indices]
    

    #### evaluating the model
    rmse = evaluate_rmse(y_train, y_pred)
    # nash-sutcliffe efficienty (NSE) - equivalent to the coefficient of determination
    nse = evaluate_nse(y_train, y_pred)
    # anomaly correlation coefficient (ACC)
    # sharpness of forecasting | ability to forecast extreme values
    mean_monthly_diversion = np.mean(y_train)
    acc = evaluate_acc(y_train, y_pred, mean_monthly_diversion)
    # mean square skill score (MSSS)
    # shows the forecaset skill by quantifying accuracy relative to the long-term monthly averged irrigation demand
    # value ranges from 0 to infinity
    # a good model will have an MSSS value closer to zero
    msss = evaluate_msss(y_train, y_pred, mean_monthly_diversion)
    # coefficient of determination
    r2 = evaluate_r2(y_train, y_pred)

    print('\tTrain: RMSE: {} | NSE: {} | ACC: {} | MSSS: {} | R-squared: {}'.format(rmse, nse, acc, msss, r2))

    ################## train set prediction ##################


    ################## test set prediction ##################
    X_test = df_feature_test[w_col + o_col]
    y_test = df_feature_test[actual_diversion_colname]

    y_pred = model.predict(X_test)
    df_feature_test[pred_diversion_colname] = np.round(y_pred, 2)

    # Find indices of NaN values in y_test or y_pred
    nan_indices = np.isnan(y_test) | np.isnan(y_pred)

    # Filter out NaN values from both y_test and y_pred
    y_test = y_test[~nan_indices]
    y_pred = y_pred[~nan_indices]


    #### evaluating the model
    rmse = evaluate_rmse(y_test, y_pred)
    # nash-sutcliffe efficienty (NSE) - equivalent to the coefficient of determination
    nse = evaluate_nse(y_test, y_pred)
    # anomaly correlation coefficient (ACC)
    # sharpness of forecasting | ability to forecast extreme values
    # mean_monthly_diversion = np.mean(y_train)
    acc = evaluate_acc(y_test, y_pred, mean_monthly_diversion)
    # mean square skill score (MSSS)
    # shows the forecaset skill by quantifying accuracy relative to the long-term monthly averged irrigation demand
    # value ranges from 0 to infinity
    # a good model will have an MSSS value closer to zero
    msss = evaluate_msss(y_test, y_pred, mean_monthly_diversion)
    # coefficient of determination
    r2 = evaluate_r2(y_test, y_pred)

    print('\tTest: RMSE: {} | NSE: {} | ACC: {} | MSSS: {} | R-squared: {}'.format(rmse, nse, acc, msss, r2))

    ################## test set predition ##################

    print()

    model = None

In [ ]:
print(df_feature_test.iloc[-1, -7:])

In [ ]:
df_feature_test